## RAG Application using TypeSense

In [1]:
import typesense

In [2]:
client=typesense.Client({
  'nodes': [{
    'host': 'ku3vwtxi1z0h7n9rp-1.a1.typesense.net',  # For Typesense Cloud use xxx.a1.typesense.net
    'port': '443',       # For Typesense Cloud use 443
    'protocol': 'https'    # For Typesense Cloud use https
  }],
  'api_key':'fncbX4EkCeoK4gGtZ3dsVoAwZI7onF1Z',
  'connection_timeout_seconds': 2
})

books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string'},
    {'name': 'authors', 'type': 'string[]', 'facet': True},
    {'name': 'publication_year', 'type': 'int32', 'facet': True},
    {'name': 'ratings_count', 'type': 'int32'},
    {'name': 'average_rating', 'type': 'float'}
  ],
  'default_sorting_field': 'ratings_count'
}
print(client.collections.create(books_schema))

{'created_at': 1762494363, 'default_sorting_field': 'ratings_count', 'enable_nested_fields': False, 'fields': [{'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'title', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'authors', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string[]'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'publication_year', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'ratings_count', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'average_rating', 'optional': False, 'sort': True, 'stem': Fal

In [3]:
with open('books.jsonl', 'r', encoding='utf-8') as jsonl_file:
    data = jsonl_file.read()
    client.collections['books'].documents.import_(data)

In [4]:
search_parameters={
    'q':"harry potter",
    'query_by':"title,authors",
    'sort_by':"ratings_count:desc"
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 17,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}},
  {'document': {'authors': ['J.K. Rowling', ' Mary GrandPré', ' R

In [5]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'publication_year:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

c:\Rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["GROQ_API_KEY"] = " pZH2RrcFCja0mPHclU8FJXqSsHqkDs6r"

In [8]:
loader = TextLoader("Statistics notes.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

C:\Users\Palak\AppData\Local\Temp\ipykernel_37952\924362634.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\Palak\AppData\Local\Temp\ipykernel_37952\924362634.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Rag\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but y

In [9]:
docsearch=Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "ku3vwtxi1z0h7n9rp-1.a1.typesense.net",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "443",  # Use 443 for Typesense Cloud
        "protocol": "https",  # Use https for Typesense Cloud
        "typesense_api_key":"pZH2RrcFCja0mPHclU8FJXqSsHqkDs6r",
        "typesense_collection_name": "lang-chain"
    },
    
)

In [11]:
query = "What is Z Test"
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

Z test is applied if population std dev is known else T test is applied.

Chi square test
It is a test that claims about population proportions.
It is a non parametric test that is performed on the categorical data(nominal or ordinal) data.

Covariance
P value < significance value --> Reject or accept null hypothesis
Cov(x,y) = sum(xi - xbar)(yi - ybar)/N

Pearson Correlation coefficient:
More the value is near 1 it is more positive correlation
The more towards -1 more it is negatively correlated
r= n(âˆ‘xy)âˆ’(âˆ‘x)(âˆ‘y)/[nâˆ‘x2âˆ’(âˆ‘x)2][nâˆ‘y2âˆ’(âˆ‘y)2]
=cov(x,y)/std dev(x).std dev(y)

Spearman rank = cov(x,y)/Rank(stddevx).Rstddev(y)
â€‹


In [12]:
### Retriever
retriever = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['Typesense', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.typesense.Typesense object at 0x0000026037884590>, search_kwargs={})

In [13]:
query = "T test"
retriever.invoke(query)[0]

Document(metadata={'source': 'Statistics notes.txt'}, page_content='Z test is applied if population std dev is known else T test is applied.\n\nChi square test\nIt is a test that claims about population proportions.\nIt is a non parametric test that is performed on the categorical data(nominal or ordinal) data.\n\nCovariance\nP value < significance value --> Reject or accept null hypothesis\nCov(x,y) = sum(xi - xbar)(yi - ybar)/N\n\nPearson Correlation coefficient:\nMore the value is near 1 it is more positive correlation\nThe more towards -1 more it is negatively correlated\nr= n(âˆ‘xy)âˆ’(âˆ‘x)(âˆ‘y)/[nâˆ‘x2âˆ’(âˆ‘x)2][nâˆ‘y2âˆ’(âˆ‘y)2]\n=cov(x,y)/std dev(x).std dev(y)\n\nSpearman rank = cov(x,y)/Rank(stddevx).Rstddev(y)\nâ€‹')